In [ ]:
import os
import time
import json
from typing import List, Dict, Any, Optional, Literal
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, Parallel, END
from langgraph.prebuilt import create_react_agent

# --- LLM & Config ---
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192") # pyright: ignore[reportCallIssue]

# --- Data Storage (ID reference) ---
class DataStorage:
    _storage = { 'transcripts': {}, 'summaries': {}, 'insights': {}, 'action_items': {} }

    @classmethod
    def store(cls, kind: str, data: Any) -> str:
        uid = f"{kind}_{int(time.time()*1000)}"
        cls._storage[kind][uid] = data
        return uid

    @classmethod
    def load(cls, kind: str, uid: str) -> Any:
        return cls._storage[kind].get(uid)

# --- Slack Tools (via MCP) ---
@tool
def slack_list_channels(limit: int = 100, cursor: Optional[str] = None) -> Dict:
    return mcp_manager.call_tool("slack_list_channels", {"limit": limit, "cursor": cursor})

@tool
def slack_post_message(channel_id: str, text: str) -> Dict:
    if len(text) > 3900:
        text = text[:3900] + "..."
    return mcp_manager.call_tool("slack_post_message", {"channel_id": channel_id, "text": text})

@tool
def slack_find_user_by_name(name: str) -> Optional[str]:
    users = slack_list_channels(limit=200).get('members', []) # pyright: ignore[reportCallIssue]
    for u in users:
        if name.lower() in u.get('real_name', '').lower():
            return u['id']
    return None

# --- Agent Prompts & Creation ---

def make_react_agent(system_prompt: str, tools: List) -> Any:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="inputs")
    ])
    return create_react_agent(model=llm, tools=tools, prompt=prompt)

# 1) Summary Agent
summary_prompt = """
You are MeetingPlus-SummaryAgent.
- Input: transcript_id
- Load transcript via DataStorage.load('transcripts', transcript_id)
- Produce a concise bullet-point summary (<300 words).
- Return JSON: {"summary": "..."}
"""
summary_agent = make_react_agent(summary_prompt, [])

# 2) Insights Agent
insights_prompt = """
You are MeetingPlus-InsightsAgent.
- Input: transcript_id
- Load transcript via DataStorage.load('transcripts', transcript_id)
- Extract key insights as list of strings.
- Identify mentioned participants by name.
- Return JSON: {"insights": [...], "users": ["Alice", "Bob"]}
"""
insights_agent = make_react_agent(insights_prompt, [])

# 3) Slack Dispatch Agent
dispatch_prompt = """
You are MeetingPlus-SlackAgent, responsible for broadcasting.
<Context>
- Inputs: summary_id, insights_id, action_items_id, topic_tags, user_map
- Tools: slack_list_channels, slack_post_message, slack_find_user_by_name
</Context>
<Responsibilities>
1. Load via DataStorage.load
2. Always post summary in 'all-abc'.
3. For each topic in topic_tags, post in '{topic}-team' if exists.
4. Format: *Summary*, *Insights*, *Action Items* with <@user_id> tags.
5. Single thread per channel.
6. Return JSON: {"dispatched": [{"channel_id":"...","ts":"..."}], "warnings": []}
</Responsibilities>
<Error Handling>
- Skip missing channels, warn.
- Retry each failure once.
</Error Handling>
<Output>Valid JSON only.</Output>
"""
dispatch_agent = make_react_agent(dispatch_prompt, [slack_list_channels, slack_post_message, slack_find_user_by_name])

@tool
def slack_dispatch(
    summary_id: str,
    insights_id: str,
    action_items_id: str,
    topic_tags: List[str],
    user_map: Dict[str, str]
) -> Dict:
    payload = {"summary_id": summary_id, "insights_id": insights_id,
               "action_items_id": action_items_id,
               "topic_tags": topic_tags, "user_map": user_map}
    result = dispatch_agent.invoke({"inputs": payload})
    return json.loads(result)

# --- StateGraph Definition ---
graph = StateGraph()

# Coordinator: ingest transcript
@graph.node()
def coordinator(transcript: str) -> Dict[str, str]:
    tid = DataStorage.store('transcripts', transcript)
    return {'transcript_id': tid}

# Parallel summary & insights
parallel = Parallel(on_success='verify')

@parallel.node()
def summarization(transcript_id: str) -> Dict[str, str]:
    out = summary_agent.invoke({'inputs': {'transcript_id': transcript_id}})
    sid = DataStorage.store('summaries', json.loads(out)['summary'])
    return {'summary_id': sid}

@parallel.node()
def insights(transcript_id: str) -> Dict[str, Any]:
    out = insights_agent.invoke({'inputs': {'transcript_id': transcript_id}})
    data = json.loads(out)
    iid = DataStorage.store('insights', data['insights'])
    # build user_map
    return {'insights_id': iid, 'user_map': {u: None for u in data['users']},
            'topic_tags': data.get('topics', [])}

graph.add_parallel(parallel, requires=['transcript_id'])

# Simple verification: ensure IDs exist
@graph.node(on_success='dispatch', on_error='error')
def verify(summary_id: str, insights_id: str) -> None:
    assert DataStorage.load('summaries', summary_id)
    assert DataStorage.load('insights', insights_id)
    return {}

# Dispatch to Slack
@graph.node(on_success=END, on_error='error')
def dispatch(
    summary_id: str,
    insights_id: str,
    action_items_id: str,
    topic_tags: List[str],
    user_map: Dict[str, str]
) -> Dict:
    return slack_dispatch(summary_id, insights_id, action_items_id, topic_tags, user_map)

# Error handler
@graph.node()
def error(**kwargs):
    # log or notify ops
    return {}

# Entry point
def run_pipeline(transcript: str):
    return graph.run({'transcript': transcript})


In [ ]:
transcript

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent

In [1]:
from typing import TypedDict, Annotated, List
import operator
from langgraph.graph import StateGraph, END
import random
import time

# 1. Define the Graph State
# This state will be passed between nodes and updated.
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        summary_status (str): Current status of the summary agent ('pending', 'success', 'failed').
        insights_status (str): Current status of the insights agent ('pending', 'success', 'failed').
        summary_output (str): The output generated by the summary agent.
        insights_output (str): The output generated by the insights agent.
        iteration (int): Tracks the number of iterations/attempts.
        error_message (str): Stores any general error message from the workflow.
    """
    summary_status: str
    insights_status: str
    summary_output: str
    insights_output: str
    iteration: int
    error_message: str

# 2. Define the Agent Nodes (Placeholders)

def run_summary_agent(state: GraphState) -> GraphState:
    """
    Simulates the execution of the summary agent.
    In a real scenario, this would call your LLM with a prompt for summarization.
    """
    print(f"\n--- Running Summary Agent (Iteration: {state['iteration']}) ---")
    new_state = state.copy()

    # Simulate success/failure for demonstration
    # For the first run, let's make it fail sometimes to test retry logic
    # After the first run, make it succeed more often
    if new_state['iteration'] == 1:
        should_fail = random.choice([True, False]) # 50% chance to fail on first attempt
    else:
        should_fail = random.random() < 0.2 # 20% chance to fail on subsequent attempts

    if should_fail:
        new_state['summary_status'] = "failed"
        new_state['summary_output'] = "Summary generation failed due to an internal error."
        print("Summary Agent: FAILED")
    else:
        new_state['summary_status'] = "success"
        new_state['summary_output'] = f"This is a generated summary from iteration {new_state['iteration']}."
        print("Summary Agent: SUCCESS")

    time.sleep(1) # Simulate work
    return new_state

def run_insights_agent(state: GraphState) -> GraphState:
    """
    Simulates the execution of the insights agent.
    In a real scenario, this would call your LLM with a prompt for insights.
    """
    print(f"\n--- Running Insights Agent (Iteration: {state['iteration']}) ---")
    new_state = state.copy()

    # Simulate success/failure for demonstration
    if new_state['iteration'] == 1:
        should_fail = random.choice([True, False]) # 50% chance to fail on first attempt
    else:
        should_fail = random.random() < 0.2 # 20% chance to fail on subsequent attempts

    if should_fail:
        new_state['insights_status'] = "failed"
        new_state['insights_output'] = "Insights generation failed due to an API timeout."
        print("Insights Agent: FAILED")
    else:
        new_state['insights_status'] = "success"
        new_state['insights_output'] = f"These are the generated insights from iteration {new_state['iteration']}."
        print("Insights Agent: SUCCESS")

    time.sleep(1) # Simulate work
    return new_state

# 3. Define the Supervisor Node (Reasoning and Decision-Making)
def decide_next_action(state: GraphState) -> str:
    """
    The supervisor agent's reasoning capability.
    It analyzes the current state and decides the next action.
    """
    print(f"\n--- Supervisor: Deciding Next Action (Iteration: {state['iteration']}) ---")

    summary_s = state['summary_status']
    insights_s = state['insights_status']

    # (1) If this is the first run, or both failed, call both in parallel
    if (state['iteration'] == 0) or \
       (summary_s == "failed" and insights_s == "failed"):
        print("Supervisor Decision: Initial run or both failed, calling both agents in parallel.")
        return "call_both_parallel"
    # (2) If only the summary agent failed in previous run, call only the summary agent individually
    elif summary_s == "failed" and insights_s == "success":
        print("Supervisor Decision: Only summary failed, calling summary agent individually.")
        return "call_summary_only"
    # (3) If only the insights agent failed, call only the insights agent individually
    elif insights_s == "failed" and summary_s == "success":
        print("Supervisor Decision: Only insights failed, calling insights agent individually.")
        return "call_insights_only"
    # (4) If both succeeded, move to end state
    elif summary_s == "success" and insights_s == "success":
        print("Supervisor Decision: Both agents succeeded, workflow complete.")
        return "end_workflow"
    else:
        # This case should ideally not be reached with the current logic,
        # but good for debugging or future expansions.
        print(f"Supervisor Decision: Unexpected state - Summary: {summary_s}, Insights: {insights_s}. Ending workflow.")
        return "end_workflow"

# A node to increment the iteration count
def increment_iteration(state: GraphState) -> GraphState:
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Incrementing Iteration to: {new_state['iteration']} ---")
    return new_state

# 4. Construct the Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("decide_next_action", decide_next_action)

# Set the entry point
workflow.set_entry_point("increment_iteration")

# Define the edges (transitions)
# After incrementing iteration, always decide next action
workflow.add_edge("increment_iteration", "decide_next_action")

# Conditional routing from the supervisor's decision node
workflow.add_conditional_edges(
    "decide_next_action",
    decide_next_action, # The function that determines the next edge
    {
        "call_both_parallel": ["run_summary_agent", "run_insights_agent"], # Call both in parallel
        "call_summary_only": "run_summary_agent",
        "call_insights_only": "run_insights_agent",
        "end_workflow": END # End the graph if both succeeded
    }
)

# After running summary agent (alone or in parallel), decide next action
workflow.add_edge("run_summary_agent", "decide_next_action")
# After running insights agent (alone or in parallel), decide next action
workflow.add_edge("run_insights_agent", "decide_next_action")


# Compile the graph
app = workflow.compile()

# --- Example Usage ---
print("--- Starting Workflow ---")

# Initial state
initial_state = {
    "summary_status": "pending",
    "insights_status": "pending",
    "summary_output": "",
    "insights_output": "",
    "iteration": 0, # Start at 0, incremented to 1 on first run
    "error_message": ""
}

# Run the graph
# The graph will run until it reaches the END state.
# You can uncomment the line below to view the graph structure
# from IPython.display import Image, display
# display(Image(app.get_graph().draw_png()))

for s in app.stream(initial_state):
    print(s)
    print("--- Current State ---")
    # LangGraph streams the state updates. We can print the latest state.
    for key, value in s.items():
        if key != '__end__': # Don't print the internal __end__ key
            print(f"{key}: {value}")
    print("---------------------\n")

# Final state after execution
final_state = next(iter(s.values())) # Get the last state yielded by the stream
print("\n--- Workflow Finished ---")
print(f"Final Summary Status: {final_state['summary_status']}")
print(f"Final Insights Status: {final_state['insights_status']}")
print(f"Final Summary Output: {final_state['summary_output']}")
print(f"Final Insights Output: {final_state['insights_output']}")
print(f"Total Iterations: {final_state['iteration']}")



TypeError: unhashable type: 'list'

In [2]:

from typing import TypedDict, Annotated, List, Literal
import operator
from langgraph.graph import StateGraph, END
import random
import time
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field

# 1. Define the Graph State
# This state will be passed between nodes and updated.
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        document_content (str): The raw meeting transaction content.
        summary_status (str): Current status of the summary agent ('pending', 'success', 'failed').
        insights_status (str): Current status of the insights agent ('pending', 'success', 'failed').
        summary_output (str): The output generated by the summary agent.
        insights_output (str): The output generated by the insights agent.
        iteration (int): Tracks the number of iterations/attempts.
        error_message (str): Stores any general error message from the workflow.
        # Removed needs_confirmation, last_agent_output_for_confirmation, user_response
        current_reasoning: str # To store the LLM's reasoning for its decision
    """
    document_content: str
    summary_status: str
    insights_status: str
    summary_output: str
    insights_output: str
    iteration: int
    error_message: str
    current_reasoning: str


# Pydantic model for the LLM's structured output
class Router(BaseModel):
    """Route the workflow to the next appropriate agent."""
    next: Literal["call_both_parallel", "call_summary_only", "call_insights_only", "end_workflow"] = Field(
        description="The name of the next node or action to route to."
    )
    reasoning: str = Field(
        description="The reasoning behind the chosen route, based on agent statuses and workflow goals."
    )

# 2. Define the Agent Nodes (Placeholders)

def run_summary_agent(state: GraphState) -> GraphState:
    """
    Simulates the execution of the summary agent.
    It uses the 'document_content' from the state as its input.
    In a real scenario, this would call your LLM with a prompt for summarization
    using state['document_content'].
    """
    print(f"\n--- Running Summary Agent (Iteration: {state['iteration']}) ---")
    print(f"Summary Agent processing content: '{state['document_content'][:50]}...'")
    new_state = state.copy()

    # Simulate success/failure
    if new_state['iteration'] == 1:
        should_fail = random.choice([True, False]) # 50% chance to fail on first attempt
    else:
        should_fail = random.random() < 0.2 # 20% chance to fail on subsequent attempts

    if should_fail:
        new_state['summary_status'] = "failed"
        new_state['summary_output'] = "Summary generation failed due to an internal error."
        print("Summary Agent: FAILED")
    else:
        new_state['summary_status'] = "success"
        new_state['summary_output'] = f"This is a generated summary from iteration {new_state['iteration']} based on: {state['document_content'][:30]}..."
        print("Summary Agent: SUCCESS")

    time.sleep(1) # Simulate work
    return new_state

def run_insights_agent(state: GraphState) -> GraphState:
    """
    Simulates the execution of the insights agent.
    It uses the 'document_content' from the state as its input.
    In a real scenario, this would call your LLM with a prompt for insights
    using state['document_content'].
    """
    print(f"\n--- Running Insights Agent (Iteration: {state['iteration']}) ---")
    print(f"Insights Agent processing content: '{state['document_content'][:50]}...'")
    new_state = state.copy()

    # Simulate success/failure
    if new_state['iteration'] == 1:
        should_fail = random.choice([True, False]) # 50% chance to fail on first attempt
    else:
        should_fail = random.random() < 0.2 # 20% chance to fail on subsequent attempts

    if should_fail:
        new_state['insights_status'] = "failed"
        new_state['insights_output'] = "Insights generation failed due to an API timeout."
        print("Insights Agent: FAILED")
    else:
        new_state['insights_status'] = "success"
        new_state['insights_output'] = f"These are the generated insights from iteration {new_state['iteration']} based on: {state['document_content'][:30]}..."
        print("Insights Agent: SUCCESS")

    time.sleep(1) # Simulate work
    return new_state

# 3. Define the Supervisor Node (Reasoning and Decision-Making with LLM)
def decide_next_action(state: GraphState) -> str:
    """
    The supervisor agent's reasoning capability, driven by an LLM.
    It analyzes the current state (summary/insights status) and decides the next action.
    """
    print(f"\n--- Supervisor: Deciding Next Action (Iteration: {state['iteration']}) ---")
    print(f"Current Statuses: Summary={state['summary_status']}, Insights={state['insights_status']}")

    # System prompt for the supervisor LLM
    system_prompt = """
    You are a highly intelligent supervisor agent tasked with managing a workflow to summarize a meeting transaction and extract key insights from it.
    Your goal is to ensure both the 'summary_agent' and 'insights_agent' successfully complete their tasks.
    You will receive the current status of these agents and must decide the next action.

    ### AVAILABLE ACTIONS:
    - 'call_both_parallel': Call both the summary and insights agents simultaneously. Use this for the initial run or if both agents previously failed.
    - 'call_summary_only': Call only the summary agent. Use this if only the summary agent failed in the previous run, and the insights agent succeeded or was not run.
    - 'call_insights_only': Call only the insights agent. Use this if only the insights agent failed in the previous run, and the summary agent succeeded or was not run.
    - 'end_workflow': Terminate the workflow. Use this only when BOTH the summary agent and the insights agent have successfully completed their tasks.

    ### CURRENT AGENT STATUSES:
    - summary_status: {summary_status} (Can be 'pending', 'success', 'failed')
    - insights_status: {insights_status} (Can be 'pending', 'success', 'failed')
    - iteration: {iteration} (Indicates how many times the workflow has attempted actions)

    ### RULES FOR DECISION-MAKING:
    1.  **Initial Run:** If it's the very first attempt (iteration 1, and both statuses are 'pending'), always start by calling both agents in parallel.
    2.  **Both Failed:** If both the summary_agent and insights_agent failed in their last attempt, retry both in parallel.
    3.  **Summary Failed Only:** If only the summary_agent failed and the insights_agent succeeded, call only the summary_agent.
    4.  **Insights Failed Only:** If only the insights_agent failed and the summary_agent succeeded, call only the insights_agent.
    5.  **All Successful:** If both the summary_agent and insights_agent have a 'success' status, the workflow is complete. Route to 'end_workflow'.
    6.  **Fallback/Unexpected State:** If the state doesn't perfectly match the above rules but the workflow is not yet complete (i.e., not both 'success'), default to retrying both in parallel to ensure progress. This acts as a safety net.

    Your response MUST be a JSON object conforming to the 'Router' schema.
    Provide clear and concise reasoning for your decision.
    """

    # Simulate LLM call
    # In a real scenario, you would use:
    # from langchain_google_genai import ChatGoogleGenerativeAI
    # groq_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    # response = groq_model.with_structured_output(Router).invoke(messages)

    # For this example, we'll simulate the LLM's decision based on the rules.
    # This simulates the LLM's reasoning based on the provided prompt.
    simulated_llm_decision = Router(next="end_workflow", reasoning="Default end.") # Default

    summary_s = state['summary_status']
    insights_s = state['insights_status']
    iteration = state['iteration']

    # Rule 1 & 2: Initial run or both failed
    if (iteration == 1 and summary_s == "pending" and insights_s == "pending") or \
       (summary_s == "failed" and insights_s == "failed"):
        simulated_llm_decision = Router(
            next="call_both_parallel",
            reasoning="Initial run or both agents failed, attempting parallel execution."
        )
    # Rule 3: Summary Failed Only
    elif summary_s == "failed" and insights_s == "success":
        simulated_llm_decision = Router(
            next="call_summary_only",
            reasoning="Only the summary agent failed, retrying it individually."
        )
    # Rule 4: Insights Failed Only
    elif insights_s == "failed" and summary_s == "success":
        simulated_llm_decision = Router(
            next="call_insights_only",
            reasoning="Only the insights agent failed, retrying it individually."
        )
    # Rule 5: All Successful
    elif summary_s == "success" and insights_s == "success":
        simulated_llm_decision = Router(
            next="end_workflow",
            reasoning="Both summary and insights agents completed successfully. Workflow finished."
        )
    # Rule 6: Fallback/Unexpected State (Not yet successful)
    else:
        simulated_llm_decision = Router(
            next="call_both_parallel",
            reasoning="Current state is ambiguous or not fully successful. Falling back to parallel retry to ensure progress."
        )

    print(f"🤖 LLM Decision: {simulated_llm_decision.next} - Reasoning: {simulated_llm_decision.reasoning}")

    new_state = state.copy()
    new_state['current_reasoning'] = simulated_llm_decision.reasoning

    # Return the decision to LangGraph
    return simulated_llm_decision.next if simulated_llm_decision.next != "end_workflow" else END


# A node to increment the iteration count
def increment_iteration(state: GraphState) -> GraphState:
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Incrementing Iteration to: {new_state['iteration']} ---")
    return new_state

# 4. Construct the Graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("decide_next_action", decide_next_action) # This is now LLM-driven

# Set the entry point
workflow.set_entry_point("increment_iteration")

# Define the edges (transitions)
# After incrementing iteration, always decide next action
workflow.add_edge("increment_iteration", "decide_next_action")

# Conditional routing from the supervisor's decision node
workflow.add_conditional_edges(
    "decide_next_action",
    decide_next_action, # The function that determines the next edge
    {
        "call_both_parallel": ["run_summary_agent", "run_insights_agent"], # Call both in parallel
        "call_summary_only": "run_summary_agent",
        "call_insights_only": "run_insights_agent",
        "end_workflow": END # End the graph if both succeeded
    }
)

# After running summary agent (alone or in parallel), decide next action
workflow.add_edge("run_summary_agent", "decide_next_action")
# After running insights agent (alone or in parallel), decide next action
workflow.add_edge("run_insights_agent", "decide_next_action")


# Compile the graph
app = workflow.compile()

# --- Example Usage ---
print("--- Starting Workflow ---")

# Initial state for a meeting transaction
initial_state = {
    "document_content": "Meeting minutes from Q3 planning session: Discussed budget allocations for marketing, R&D, and operations. Key decisions include increasing marketing spend by 15% for digital campaigns, freezing R&D hiring for Q4, and optimizing operational costs by 10% through vendor renegotiations. Action items: Marketing team to draft new campaign proposals by EOD Friday. Finance to provide updated budget forecasts by next Monday. Next meeting: October 15th.",
    "summary_status": "pending",
    "insights_status": "pending",
    "summary_output": "",
    "insights_output": "",
    "iteration": 0, # Start at 0, incremented to 1 on first run
    "error_message": "",
    "current_reasoning": ""
}

# Run the graph
# The graph will run until it reaches the END state.
# from IPython.display import Image, display
# display(Image(app.get_graph().draw_png())) # Uncomment to visualize graph

for s in app.stream(initial_state):
    print(s)
    print("--- Current State ---")
    # LangGraph streams the state updates. We can print the latest state.
    for key, value in s.items():
        if key != '__end__': # Don't print the internal __end__ key
            print(f"{key}: {value}")
    print("---------------------\n")

# Final state after execution
final_state = next(iter(s.values())) # Get the last state yielded by the stream
print("\n--- Workflow Finished ---")
print(f"Final Summary Status: {final_state['summary_status']}")
print(f"Final Insights Status: {final_state['insights_status']}")
print(f"Final Summary Output: {final_state['summary_output']}")
print(f"Final Insights Output: {final_state['insights_output']}")
print(f"Total Iterations: {final_state['iteration']}")



/home/pavan/Ds/pro/smart-meeting-copilot/smartcopilot-api/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


TypeError: unhashable type: 'list'

In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
import random
import time
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field


# Keep the original simple state - just add what's needed
class GraphState(TypedDict):
    document_content: str
    summary_status: str
    insights_status: str
    summary_output: str
    insights_output: str
    iteration: int
    error_message: str
    current_reasoning: str

# Simple but intelligent decision model
class SupervisorDecision(BaseModel):
    """AI Supervisor decision with intelligent reasoning"""
    next_action: Literal[
        "call_both_parallel", 
        "call_summary_only", 
        "call_insights_only", 
        "end_workflow"
    ] = Field(description="What to do next in the workflow")
    
    reasoning: str = Field(description="Why this decision makes sense for the workflow goal")
    
    confidence: float = Field(
        description="Confidence in this decision (0.0 to 1.0)", 
        ge=0.0, le=1.0
    )

def intelligent_supervisor(state: GraphState) -> str:
    """
    Smart supervisor that understands the workflow goal:
    - Get meeting minutes summary 
    - Get key insights in parallel
    - Be intelligent about when to retry vs when to finish
    """
    print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
    # System prompt focused on workflow intelligence, not error handling
    system_prompt = """
    You are an intelligent workflow supervisor. Your ONLY goal is to ensure we get:
    1. A good summary of the meeting minutes
    2. Key insights from the meeting minutes  
    3. Both should be done efficiently
    
    You need to be SMART about:
    - When something failed due to temporary issues (network, API busy) → retry makes sense
    - When we have what we need → finish the workflow  
    - When to run agents in parallel vs individually based on current state
    - When we've tried enough and should accept current results
    
    WORKFLOW CONTEXT:
    - Meeting document: "{doc_preview}"
    - Summary status: {summary_status} 
    - Insights status: {insights_status}
    - Current iteration: {iteration}
    - Last error: {error_msg}
    
    THINK INTELLIGENTLY:
    - If both are successful → we're done!
    - If it's iteration 1 → start both in parallel (efficient)
    - If one failed and other succeeded → retry just the failed one
    - If both failed but iteration is low → try both again (might be temporary issue)
    - If we've tried many times → maybe accept partial results or finish
    - Consider: is this a workflow issue or just temporary network/API hiccup?
    
    Be smart, not just rule-based. Think about the actual goal: getting meeting summary + insights.
    """
    
    # Format the prompt with current state
    doc_preview = state['document_content'][:100] + "..." if len(state['document_content']) > 100 else state['document_content']
    
    formatted_prompt = system_prompt.format(
        doc_preview=doc_preview,
        summary_status=state['summary_status'],
        insights_status=state['insights_status'], 
        iteration=state['iteration'],
        error_msg=state['error_message'] or "None"
    )
    
    # TODO: Replace with actual LLM call
    """
    # With Qwen or other reasoning model:
    from langchain_community.llms import Ollama
    llm = Ollama(model="qwen2.5:14b")  # or qwen2.5:32b for even better reasoning
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Analyze the workflow state and decide what to do next. Think step by step about our goal: meeting summary + insights.")
    ]
    
    decision = llm.with_structured_output(SupervisorDecision).invoke(messages)
    """
    
    # Simulate intelligent decision (replace with real LLM)
    decision = simulate_smart_decision(state)
    
    print(f"💭 AI Reasoning: {decision.reasoning}")
    print(f"⚡ Decision: {decision.next_action} (confidence: {decision.confidence:.2f})")
    
    # Update state with AI reasoning
    new_state = state.copy()
    new_state['current_reasoning'] = decision.reasoning
    
    return decision.next_action if decision.next_action != "end_workflow" else END



def simulate_smart_decision(state: GraphState) -> SupervisorDecision:
    """
    Simulate what an intelligent model like Qwen would decide
    This is just simulation - replace with actual LLM call
    """
    
    summary_status = state['summary_status']
    insights_status = state['insights_status']
    iteration = state['iteration']
    
    # Smart decision making that a good reasoning model would do
    
    # Goal achieved - both successful
    if summary_status == "success" and insights_status == "success":
        return SupervisorDecision(
            next_action="end_workflow",
            reasoning="Perfect! Both summary and insights are ready. Our workflow goal is complete - we have meeting summary + key insights as requested.",
            confidence=1.0
        )
    
    # First iteration - start efficiently  
    if iteration == 1 and summary_status == "pending" and insights_status == "pending":
        return SupervisorDecision(
            next_action="call_both_parallel", 
            reasoning="First attempt - running both summary and insights agents in parallel for efficiency. This is the optimal starting strategy.",
            confidence=0.9
        )
    
    # One succeeded, one failed - targeted retry
    if summary_status == "success" and insights_status == "failed":
        if iteration <= 3:  # Smart about retry limits
            return SupervisorDecision(
                next_action="call_insights_only",
                reasoning="Summary is ready, but insights failed. Retrying only insights agent since summary is already successful. Efficient targeted approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Summary is ready and we've tried insights multiple times. Sometimes partial success is acceptable for meeting processing workflow.",
                confidence=0.6
            )
    
    if insights_status == "success" and summary_status == "failed":
        if iteration <= 3:
            return SupervisorDecision(
                next_action="call_summary_only", 
                reasoning="Insights are ready, but summary failed. Retrying only summary agent since insights are already successful. Focused retry approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Insights are ready and we've tried summary multiple times. We have key insights from the meeting which provides value.",
                confidence=0.6
            )
    
    # Both failed - intelligent retry decision
    if summary_status == "failed" and insights_status == "failed":
        if iteration <= 2:
            return SupervisorDecision(
                next_action="call_both_parallel",
                reasoning="Both agents failed, but it's early in the process. Likely a temporary issue (network/API). Retrying both in parallel - efficient recovery approach.",
                confidence=0.7
            )
        elif iteration <= 4:
            return SupervisorDecision(
                next_action="call_both_parallel", 
                reasoning="Multiple failures but still within reasonable retry range. The meeting document seems valid, so this might be temporary service issues. One more parallel attempt.",
                confidence=0.5
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="After multiple attempts, continuing may not be productive. This could be a deeper issue with the document format or service availability. Ending workflow.",
                confidence=0.8
            )
    
    # Default intelligent fallback
    return SupervisorDecision(
        next_action="call_both_parallel",
        reasoning="Current state requires both agents to run. Taking parallel approach for efficiency in meeting processing workflow.", 
        confidence=0.6
    )

# Keep your original agent functions - they're fine
def run_summary_agent(state: GraphState) -> GraphState:
    """Summary agent - focused on meeting summary"""
    print(f"\n📝 Summary Agent working on meeting minutes...")
    new_state = state.copy()
    
    # Simulate work with some intelligence about content
    success_rate = 0.7 if "meeting" in state['document_content'].lower() else 0.5
    
    if random.random() < success_rate:
        new_state['summary_status'] = "success"
        new_state['summary_output'] = f"Meeting Summary: Key decisions and action items extracted from meeting minutes (iteration {state['iteration']})"
        print("✅ Summary: Generated meeting summary successfully")
    else:
        new_state['summary_status'] = "failed" 
        new_state['summary_output'] = "Summary generation failed"
        new_state['error_message'] = "Temporary API issue during summary generation"
        print("❌ Summary: Failed (likely temporary issue)")
    
    time.sleep(1)
    return new_state

def run_insights_agent(state: GraphState) -> GraphState:
    """Insights agent - focused on meeting insights"""
    print(f"\n🔍 Insights Agent extracting key insights...")
    new_state = state.copy()
    
    # Simulate work with some intelligence about content  
    success_rate = 0.7 if "meeting" in state['document_content'].lower() else 0.5
    
    if random.random() < success_rate:
        new_state['insights_status'] = "success"
        new_state['insights_output'] = f"Meeting Insights: Key themes, decisions, and follow-ups identified (iteration {state['iteration']})"
        print("✅ Insights: Extracted meeting insights successfully")
    else:
        new_state['insights_status'] = "failed"
        new_state['insights_output'] = "Insights generation failed"  
        new_state['error_message'] = "Temporary processing issue during insights extraction"
        print("❌ Insights: Failed (likely temporary issue)")
    
    time.sleep(1)
    return new_state

def increment_iteration(state: GraphState) -> GraphState:
    """Track iterations for intelligent decision making"""
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Workflow Iteration: {new_state['iteration']} ---")
    return new_state

# Build the workflow graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("intelligent_supervisor", intelligent_supervisor)  # This is the key AI decision maker
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)

# Set entry point
workflow.set_entry_point("increment_iteration")

# Define flow
workflow.add_edge("increment_iteration", "intelligent_supervisor")

# The AI supervisor makes intelligent routing decisions
workflow.add_conditional_edges(
    "intelligent_supervisor",
    intelligent_supervisor,
    {
        "call_both_parallel": ["run_summary_agent", "run_insights_agent"],
        "call_summary_only": "run_summary_agent", 
        "call_insights_only": "run_insights_agent",
        "end_workflow": END
    }
)

# After each agent, go back to supervisor for next intelligent decision
workflow.add_edge("run_summary_agent", "intelligent_supervisor")
workflow.add_edge("run_insights_agent", "intelligent_supervisor")

# Compile
app = workflow.compile()

# Test the intelligent workflow
print("🚀 Starting Intelligent Meeting Processing Workflow")

initial_state = {
    "document_content": "Meeting minutes from Q3 planning: Budget discussion, marketing strategy changes, team restructuring decisions, action items for next quarter...",
    "summary_status": "pending",
    "insights_status": "pending", 
    "summary_output": "",
    "insights_output": "",
    "iteration": 0,
    "error_message": "",
    "current_reasoning": ""
}

# Run and see the AI make intelligent decisions
for step in app.stream(initial_state):
    if '__end__' not in step:
        print(f"\n📊 Current State: {step}")

print("\n🎯 Workflow completed with AI supervision!")

TypeError: unhashable type: 'list'

In [74]:
#working

In [1]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
import random
import time
import os
from langgraph.types import Command
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent


In [2]:
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")



In [3]:

# Keep the original simple state - just add what's needed
class GraphState(TypedDict):
    document_content: str
    summary_status: str
    insights_status: str
    summary_output: str
    insights_output: str
    iteration: int
    error_message: str
    current_reasoning: str
    next : str

In [4]:

# Simple but intelligent decision model
class SupervisorDecision(BaseModel):
    """AI Supervisor decision with intelligent reasoning"""
    next_action: Literal[
        "call_both_parallel", 
        "call_summary_only", 
        "call_insights_only", 
        "end_workflow"
    ] = Field(description="What to do next in the workflow")
    
    reasoning: str = Field(description="Why this decision makes sense for the workflow goal")
    
    confidence: float = Field(
        description="Confidence in this decision (0.0 to 1.0)", 
        ge=0.0, le=1.0
    )

In [5]:
from typing import Dict, Any
def intelligent_supervisor(state: GraphState) ->  Dict[str, Any]:
    """
    Smart supervisor that understands the workflow goal:
    - Get meeting minutes summary 
    - Get key insights in parallel
    - Be intelligent about when to retry vs when to finish
    """
    print(f"\n🧠 AI Supervisor thinking... (Iteration: {state['iteration']})")
    
    # Enhanced ReAct-style system prompt
    system_prompt = """
    You are an intelligent workflow supervisor using ReAct (Reasoning + Acting) methodology. 

    ## YOUR MISSION:
    Ensure we successfully get:
    1. A good SUMMARY of the meeting minutes
    2. KEY INSIGHTS from the meeting minutes
    3. Both delivered EFFICIENTLY

    ## ReAct PROCESS - Think step by step:

    **THOUGHT**: First, analyze the current workflow state. What's working? What failed? Why might it have failed?

    **OBSERVATION**: What do you observe about the current status? Look at:
    - What agents have succeeded/failed
    - How many iterations we've done  
    - Whether failures seem temporary (network/API issues) or persistent
    - If we have partial success that might be sufficient

    **ACTION**: Based on your thought and observation, decide the smartest next action for our meeting processing goal.

    ## CURRENT WORKFLOW STATE:
    - Meeting Document: "{doc_preview}"
    - Summary Status: {summary_status}
    - Insights Status: {insights_status} 
    - Current Iteration: {iteration}
    - Last Error: {error_msg}

    ## AVAILABLE ACTIONS:
    - call_both_parallel: Run both agents simultaneously (efficient for fresh start or when both need work)
    - call_summary_only: Focus only on getting the meeting summary
    - call_insights_only: Focus only on getting meeting insights  
    - end_workflow: We have achieved our goal (both summary + insights ready)

    ## INTELLIGENT DECISION GUIDELINES:
    - Iteration 1: Usually start with parallel execution for efficiency
    - One success, one failure: Target retry the failed agent only
    - Both failed early iterations: Likely temporary issues, retry both
    - Multiple failures: Consider if we should accept partial results or continue
    - Both successful: Mission accomplished!

    **Remember**: You're optimizing for getting useful meeting analysis (summary + insights), not perfect success rates.

    Use the ReAct process: THOUGHT → OBSERVATION → ACTION with clear reasoning.
    """
    
    # Format the prompt with current state
    doc_preview = state['document_content'][:100] + "..." if len(state['document_content']) > 100 else state['document_content']
    
    formatted_prompt = system_prompt.format(
        doc_preview=doc_preview,
        summary_status=state['summary_status'],
        insights_status=state['insights_status'], 
        iteration=state['iteration'],
        error_msg=state['error_message'] or "None"
    )
    
    # Setup Groq API with Qwen model
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")
    
    os.environ["GROQ_API_KEY"] = groq_api_key
    
    llm = ChatGroq(
        temperature=0,
        model="deepseek-r1-distill-llama-70b"  
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Use ReAct methodology: THOUGHT → OBSERVATION → ACTION. Analyze the workflow state and decide what to do next for our meeting processing goal. Think step by step.")
    ]
    
    # Get structured decision from LLM
    try:
        decision = llm.with_structured_output(SupervisorDecision).invoke(messages)
    except Exception as e:
        print(f"⚠️ LLM call failed: {e}")
        # Fallback to simulation if LLM fails
        decision = simulate_smart_decision(state)
    
    # # Simulate intelligent decision (replace with real LLM)
    # decision = simulate_smart_decision(state)
    
    print(f"💭 AI Reasoning: {decision.reasoning}")
    print(f"⚡ Decision: {decision.next_action} (confidence: {decision.confidence:.2f})")
    
    # # Update state with AI reasoning
    # new_state = state.copy()
    # new_state['next'] = decision.next_action  # Store the next action
    # new_state['current_reasoning'] = decision.reasoning
    
    # return decision.next_action if decision.next_action != "end_workflow" else END
    goto = decision.next_action if decision.next_action != "end_workflow" else END

    print('goto:',goto)

    return {
        "current_reasoning": decision.reasoning,
        "next": decision.next_action # Store the determined next action
    }

    # return Command(
    #         # goto=goto if goto != "FINISH" else "__end__",
    #         goto = goto,
    #         update={
    #             "next": goto,
                
    #             "current_reasoning": decision.reasoning,
    #             # "messages": updated_messages  # Use updated messages
    #         }
    #     ) # pyright: ignore[reportReturnType]
        

In [6]:


def simulate_smart_decision(state: GraphState) -> SupervisorDecision:
    """
    Simulate what an intelligent model like Qwen would decide
    This is just simulation - replace with actual LLM call
    """
    
    summary_status = state['summary_status']
    insights_status = state['insights_status']
    iteration = state['iteration']
    
    # Smart decision making that a good reasoning model would do
    
    # Goal achieved - both successful
    if summary_status == "success" and insights_status == "success":
        return SupervisorDecision(
            next_action="end_workflow",
            reasoning="Perfect! Both summary and insights are ready. Our workflow goal is complete - we have meeting summary + key insights as requested.",
            confidence=1.0
        )
    
    # First iteration - start efficiently  
    if iteration == 1 and summary_status == "pending" and insights_status == "pending":
        return SupervisorDecision(
            next_action="call_both_parallel", 
            reasoning="First attempt - running both summary and insights agents in parallel for efficiency. This is the optimal starting strategy.",
            confidence=0.9
        )
    
    # One succeeded, one failed - targeted retry
    if summary_status == "success" and insights_status == "failed":
        if iteration <= 3:  # Smart about retry limits
            return SupervisorDecision(
                next_action="call_insights_only",
                reasoning="Summary is ready, but insights failed. Retrying only insights agent since summary is already successful. Efficient targeted approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Summary is ready and we've tried insights multiple times. Sometimes partial success is acceptable for meeting processing workflow.",
                confidence=0.6
            )
    
    if insights_status == "success" and summary_status == "failed":
        if iteration <= 3:
            return SupervisorDecision(
                next_action="call_summary_only", 
                reasoning="Insights are ready, but summary failed. Retrying only summary agent since insights are already successful. Focused retry approach.",
                confidence=0.8
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="Insights are ready and we've tried summary multiple times. We have key insights from the meeting which provides value.",
                confidence=0.6
            )
    
    # Both failed - intelligent retry decision
    if summary_status == "failed" and insights_status == "failed":
        if iteration <= 2:
            return SupervisorDecision(
                next_action="call_both_parallel",
                reasoning="Both agents failed, but it's early in the process. Likely a temporary issue (network/API). Retrying both in parallel - efficient recovery approach.",
                confidence=0.7
            )
        elif iteration <= 4:
            return SupervisorDecision(
                next_action="call_both_parallel", 
                reasoning="Multiple failures but still within reasonable retry range. The meeting document seems valid, so this might be temporary service issues. One more parallel attempt.",
                confidence=0.5
            )
        else:
            return SupervisorDecision(
                next_action="end_workflow",
                reasoning="After multiple attempts, continuing may not be productive. This could be a deeper issue with the document format or service availability. Ending workflow.",
                confidence=0.8
            )
    
    # Default intelligent fallback
    return SupervisorDecision(
        next_action="call_both_parallel",
        reasoning="Current state requires both agents to run. Taking parallel approach for efficiency in meeting processing workflow.", 
        confidence=0.6
    )


In [7]:
def route_supervisor_decision(state: GraphState) -> str:
    """
    Routes the workflow based on the decision stored by the intelligent supervisor.
    """
    return state['next'] 

In [8]:

# # Keep your original agent functions - they're fine
# def run_summary_agent(state: GraphState) -> GraphState:
#     """Summary agent - focused on meeting summary"""
#     print(f"\n📝 Summary Agent working on meeting minutes...")
#     new_state = state.copy()
    
#     # Simulate work with some intelligence about content
#     success_rate = 0.1 if "meeting" in state['document_content'].lower() else 0.3
    
#     if random.random() < success_rate:
#         new_state['summary_status'] = "success"
#         new_state['summary_output'] = f"Meeting Summary: Key decisions and action items extracted from meeting minutes (iteration {state['iteration']})"
#         print("✅ Summary: Generated meeting summary successfully")
#     else:
#         new_state['summary_status'] = "failed" 
#         new_state['summary_output'] = "Summary generation failed"
#         new_state['error_message'] = "Temporary API issue during summary generation"
#         print("❌ Summary: Failed (likely temporary issue)")
    
#     time.sleep(1)
#     return new_state



In [9]:

def run_summary_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently generates a meeting summary.
    It has a specific persona, instructions, and robust error handling.
    """
    print("\n🤖 Agentic Summary Node Called...")

    # 1. Define the Agent's Persona and Mission via a System Prompt
    system_prompt = """
    You are an expert Meeting Summarization Agent. Your sole mission is to create a concise, structured, and insightful summary from the provided meeting minutes.

    ## Your Guidelines:
    1.  **Identify Core Content**: Focus on extracting the most critical information:
        - **Key Decisions Made**: What was formally decided?
        - **Action Items**: What are the specific next steps? Who is responsible (owner)? What are the deadlines?
        - **Major Topics Discussed**: Briefly mention the main subjects of conversation.
        - **Outcomes & Resolutions**: What was the final result of the discussions?

    2.  **Prioritize Significance**: Do not just list topics in order. Your value is in identifying what truly matters. A brief 2-minute decision that sets the company's direction is more important than a 30-minute unresolved debate.

    3.  **Structure the Output**: Present the summary in a clean, professional format using Markdown. Use headings (#), subheadings (##), and bullet points (-) for clarity.

    4.  **Be Objective**: Summarize what was said and decided without adding your own opinions or interpretations. Stick to the facts presented in the document.

    Your final output should be ONLY the structured summary, ready to be shared with meeting attendees.
    """

    # 2. Setup the Agent
    # For this task, the agent doesn't need external tools. Its "tool" is its own
    # reasoning capability applied to the text. We use the ReAct agent structure
    # for its robust reasoning loop, even with an empty tool list.
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Please generate a summary for the following meeting minutes:\n\n---\n\n{{document_content}}")
    ])

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    llm = ChatGroq(temperature=0, model="qwen/qwen3-32b")

    # The agent uses the LLM and prompt, with no external tools needed for this task.
    summary_agent = create_react_agent(llm, tools=[], prompt=prompt)

    # 3. Invoke the Agent with Error Handling
    try:
        print("🧠 Agent is thinking and generating the summary...")
        # The input to the agent is the document content from our graph state
        agent_input = {"meeting_minutes": state['document_content']}
        result = summary_agent.invoke(agent_input)

        # print("result:", result['messages'][-1].content)

        # The agent's final answer is in the last message of the result
        generated_summary = result['messages'][-1].content
        print("✅ Summary Agent: Generated summary successfully.")
        
        # Return a dictionary to update the graph's state
        return {
            "summary_status": "success",
            "summary_output": generated_summary,
            "error_message": ""  # Clear any previous error message
        }

    except Exception as e:
        print(f"❌ Error in Agentic Summary Node: {e}")
        
        # If the agent fails, report the failure back to the graph state
        # The supervisor can then decide how to handle this failure.
        return {
            "summary_status": "failed",
            "summary_output": "Summary generation failed due to an internal agent error.",
            "error_message": f"Agent Error: {str(e)}"
        }

In [10]:

# def run_insights_agent(state: GraphState) -> dict:
#     """
#     Insights agent - focused on meeting insights.
#     FIX: Now returns a dictionary and clears the error message on success.
#     """
#     print(f"\n🔍 Insights Agent extracting key insights...")
    
#     # Simulate work
#     success_rate = 0.6 if "meeting" in state['document_content'].lower() else 0.3
#     time.sleep(1)

#     if random.random() < success_rate:
#         print("✅ Insights: Extracted meeting insights successfully")
#         # On success, return a success status AND an empty error message
#         return {
#             "insights_status": "success",
#             "insights_output": f"Meeting Insights: Key themes and decisions identified (iteration {state['iteration']})",
#             "error_message": ""
#         }
#     else:
#         print("❌ Insights: Failed (simulating temporary issue)")
#         # On failure, return a failed status AND a descriptive error message
#         return {
#             "insights_status": "failed",
#             "insights_output": "Insights generation failed",
#             "error_message": "Temporary processing issue during insights extraction"
#         }

In [26]:
def run_insights_agent(state: GraphState) -> dict:
    """
    An agentic node that intelligently extracts key insights from meeting minutes.
    It has a specific persona, instructions, and robust error handling.
    """
    print("\n🤖 Agentic Insights Node Called...")

    # 1. Define the Agent's Persona and Mission via a System Prompt
    system_prompt = """
    You are an expert Meeting Insights Agent. Your mission is to extract not just facts, but the underlying meaning and implications from the provided meeting minutes.

    ## Your Guidelines:
    1. **Identify Key Themes**: Go beyond topics. What are the recurring ideas, concerns, or strategic directions? (e.g., "A recurring theme was the concern over budget constraints affecting timelines.")
    2. **Highlight Critical Decisions & Implications**: State the decision and then explain *why it matters*. (e.g., "Decision: Approved 15% marketing budget increase. Implication: This signals a strategic shift towards aggressive market capture for the new product.")
    3. **Surface Actionable Insights**: What can the team learn or do differently based on the discussion? These are not the same as action items. (e.g., "Insight: The debate on resource allocation suggests a lack of clarity in departmental priorities, which should be addressed.")
    4. **Structure the Output**: Present insights in a clear, professional format using Markdown. Use headings and bullet points.

    Your final output should be ONLY the structured insights, ready for strategic review.
    """

    # 2. Setup the Agent
    # TODO: The variable in the prompt must match the key used in the 'invoke' call.
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Please extract key insights from the following meeting minutes:\n\n---\n\n{{document_content}}")
    ])

    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    llm = ChatGroq(temperature=0.1, model="llama3-70b-8192")

    # The agent uses the LLM and prompt, with no external tools needed for this task.
    insights_agent = create_react_agent(llm, tools=[], prompt=prompt)

    # 3. Invoke the Agent with Error Handling
    try:
        print("🧠 Agent is thinking and extracting insights...")
        agent_input = {"meeting_minutes": state['document_content']}
        result = insights_agent.invoke(agent_input)

        # The agent's final answer is in the last message of the result
        generated_insights = result['messages'][-1].content
        print("✅ Insights Agent: Extracted insights successfully.")
        
        # Return a dictionary to update the graph's state
        return {
            "insights_status": "success",
            "insights_output": generated_insights,
            "error_message": ""  # Clear any previous error message
        }

    except (KeyError, IndexError, Exception) as e:
        print(f"❌ Error in Agentic Insights Node: {e}")
        
        # If the agent fails, report the failure back to the graph state
        return {
            "insights_status": "failed",
            "insights_output": "Insights generation failed due to an internal agent error.",
            "error_message": f"Agent Error: {str(e)}"
        }


In [27]:
from langchain_core.runnables import RunnableParallel

def run_both_parallel_agents(state: GraphState) -> dict:
    """
    Runs summary and insights agents in parallel.
    This now calls both of the new agentic nodes.
    """
    print("\n---RUNNING BOTH AGENTIC NODES IN PARALLEL---")

    parallel_runnable = RunnableParallel(
        summary_result=run_summary_agent,
        insights_result=run_insights_agent 
    )

    parallel_results = parallel_runnable.invoke(state)

    summary_updates = parallel_results['summary_result']
    insights_updates = parallel_results['insights_result']

    combined_updates = {**summary_updates, **insights_updates}

    summary_error = summary_updates.get("error_message", "")
    insights_error = insights_updates.get("error_message", "")

    if summary_error and insights_error:
        combined_updates["error_message"] = f"Summary Error: {summary_error} | Insights Error: {insights_error}"
    elif summary_error:
        combined_updates["error_message"] = summary_error
    elif insights_error:
        combined_updates["error_message"] = insights_error
    else:
        combined_updates["error_message"] = ""

    return combined_updates

In [28]:

# def run_both_parallel_agents(state: GraphState) -> dict:
#     """
#     Runs summary and insights agents in parallel.
#     FIX: Now correctly combines results and manages the error message.
#     """
#     print("\n---RUNNING BOTH AGENTS IN PARALLEL---")

#     # Define the parallel execution. We use the corrected insights agent.
#     # Note: run_summary_agent_agentic is your more robust agent.
#     parallel_runnable = RunnableParallel(
#         summary_result=run_summary_agent,
#         insights_result=run_insights_agent
#     )

#     # Execute in parallel
#     parallel_results = parallel_runnable.invoke(state)

#     summary_updates = parallel_results['summary_result']
#     insights_updates = parallel_results['insights_result']

#     # Combine the updates from both parallel runs
#     combined_updates = {**summary_updates, **insights_updates}

#     # --- Logic to handle combined error messages ---
#     # If both succeeded, the error message from both will be "", which is fine.
#     # If one fails, we want to preserve its specific error message.
#     summary_error = summary_updates.get("error_message", "")
#     insights_error = insights_updates.get("error_message", "")

#     if summary_error and insights_error:
#         combined_updates["error_message"] = f"Summary Error: {summary_error} | Insights Error: {insights_error}"
#     elif summary_error:
#         combined_updates["error_message"] = summary_error
#     elif insights_error:
#         combined_updates["error_message"] = insights_error
#     else:
#         combined_updates["error_message"] = "" # Both succeeded, clear error.

#     return combined_updates



In [29]:
def increment_iteration(state: GraphState) -> GraphState:
    """Track iterations for intelligent decision making"""
    new_state = state.copy()
    new_state['iteration'] += 1
    print(f"\n--- Workflow Iteration: {new_state['iteration']} ---")
    return new_state

In [30]:
# Build the workflow graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("increment_iteration", increment_iteration)
workflow.add_node("intelligent_supervisor", intelligent_supervisor)
workflow.add_node("run_summary_agent", run_summary_agent)
workflow.add_node("run_insights_agent", run_insights_agent)
workflow.add_node("run_parallel_agents", run_both_parallel_agents)

In [31]:

# Set entry point
workflow.set_entry_point("increment_iteration")

# Define initial edge from entry point to supervisor
workflow.add_edge("increment_iteration", "intelligent_supervisor")

In [32]:
# The AI supervisor makes intelligent routing decisions
# - The edge starts from the 'intelligent_supervisor' node.
# - The 'route_supervisor_decision' function defines how to choose the next path.
workflow.add_conditional_edges(
    "intelligent_supervisor",     # Conditional edges originate from the 'intelligent_supervisor' node
    route_supervisor_decision,    # This function defines the routing logic (returns the key for the dict)
    {
        "call_both_parallel": "run_parallel_agents", # Map decision to the parallel execution node
        "call_insights_only": "run_insights_agent",  # Map decision to the insights agent node
        "call_summary_only": "run_summary_agent",    # Map decision to the summary agent node
        "end_workflow": END                          # Map decision to the END state
    }
)


In [33]:
# After each agent completes, return to the supervisor for the next intelligent decision
workflow.add_edge("run_summary_agent", "intelligent_supervisor")
workflow.add_edge("run_insights_agent", "intelligent_supervisor")
workflow.add_edge("run_parallel_agents", "intelligent_supervisor") # Add this edge for parallel execution


In [34]:
# Compile the workflow
app = workflow.compile()


In [35]:

# Test the intelligent workflow
print("🚀 Starting Intelligent Meeting Processing Workflow")

initial_state = GraphState(
        document_content="Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.",
        summary_status="pending",
        insights_status="pending",
        summary_output="",
        insights_output="",
        iteration=0,
        error_message="",
        current_reasoning="",
        next=""
    )

# Run and see the AI make intelligent decisions
for step in app.stream(initial_state):
    if '__end__' not in step:
        print(f"\n📊 Current State: {step}")

print("\n🎯 Workflow completed with AI supervision!")

🚀 Starting Intelligent Meeting Processing Workflow

--- Workflow Iteration: 1 ---

📊 Current State: {'increment_iteration': {'document_content': "Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.", 'summary_status': 'pending', 'insights_status': 'pending', 'summary_output': '', 'insights_output': '', 'iteration': 1, 'error_message': '', 'current_reasoning': '', 'next': ''}}

🧠 AI Supervisor thinking... (Iteration: 1)
💭 AI Reasoning: Since it's the first iteration and both summary and insights are pending, running both agents in parallel will efficiently obtain both results simultaneously.
⚡ Decision: call_both_pa

In [ ]:
if __name__ == '__main__':
#     # This is a simple test to show how the node works in isolation
    test_state = GraphState(
        document_content="Meeting Minutes - July 21, 2025\nAttendees: Alice, Bob, Charlie\n1. Project Phoenix Update: Bob confirmed the new server is deployed. Alice raised a concern about the budget overrun. It was decided to review the Q3 budget next week. Action Item: Charlie to schedule a budget review meeting by Friday, July 25th.\n2. Marketing Campaign: Discussed the new 'Summer Sale' campaign. It will launch on August 1st. All assets are ready.",
        summary_status="pending",
        insights_status="pending",
        summary_output="",
        insights_output="",
        iteration=0,
        error_message="",
        current_reasoning="",
        next=""
    )

    result_update = run_summary_agent_agentic(test_state)
    print("\n--- TEST RESULT ---")
    import json
    print(json.dumps(result_update, indent=2))